In [21]:
import pandas as pd
import numpy as np
import utils

In [22]:
df = pd.read_json("data_no_bool.json").T

In [23]:
df.head()

,label,time_series
0,6,"[{'BallAcceleration': 1636.7987723122642, 'Dis..."
1,-1,"[{'BallAcceleration': 0.0, 'DistanceWall': 39...."
2,-1,"[{'BallAcceleration': 124246.29375405672, 'Dis..."
3,-1,"[{'BallAcceleration': -8210.634011562943, 'Dis..."
4,-1,"[{'BallAcceleration': 1197.5360615055833, 'Dis..."


In [24]:
df.time_series = df.time_series.apply(utils.extract_matrix)
df['shape'] = df.time_series.apply(lambda x: x.shape)
df.head()

,label,time_series,shape
0,6,"[[1636.7987723122642, 3498.01, 2012.98, 299.66...","(32, 17)"
1,-1,"[[0.0, 39.7800000000002, 1704.5, 193.421147499...","(39, 17)"
2,-1,"[[124246.29375405672, 35.48999999999978, 1450....","(34, 17)"
3,-1,"[[-8210.634011562943, 36.0, 1036.28, 579.43975...","(23, 17)"
4,-1,"[[1197.5360615055833, 3975.02, 2013.04, 142.93...","(22, 17)"


In [25]:
S, v_t_list = utils.compute_S_matrix(df.time_series)

In [26]:
w = utils.compute_weight_vector(S, aggregation='mean', algorithm='1')
w

array([7.33901974e-01, 2.11399585e-01, 5.46184777e-02, 6.73521064e-05,
       1.01007818e-05, 2.51086069e-06, 2.46274980e-11, 1.48000471e-11,
       9.75667933e-12, 6.57018216e-12, 4.07295441e-12, 2.01973052e-12,
       8.18530946e-13, 2.46108301e-13, 7.03280356e-14, 2.61036403e-14,
       5.54693561e-15])

Eros norm:
$$
EROS(\mathbf{A,B},w) = \sum_{i=1}^n w_i\cdot|<a_i,b_i>|
$$

In [27]:
eros = utils.eros_norm(w, A=v_t_list[0], B=v_t_list[0])
eros

0.9999999999999998

In [28]:
K_eros = utils.compute_kernel_matrix(len(df), weight_vector=w, v_t_list=v_t_list)

In [29]:
K_eros[0,1] == K_eros[1,0]

True

In [30]:
eig_vecs = utils.perform_PCA(len(df), weight_vector=w, v_t_list=v_t_list)

In [31]:
df = pd.read_json("data_no_bool.json").T
df.head()

#from sklearn.utils import shuffle
import sklearn as sk
from sklearn.model_selection import train_test_split
seed = 0

# Shuffle the dataframe
shuffled_df = sk.utils.shuffle(df, random_state=seed)
shuffled_df.head()

from utils import extract_matrix
# Create the matrix version
shuffled_df.time_series = shuffled_df.time_series.apply(extract_matrix)
shuffled_df["shape"] = shuffled_df.time_series.apply(lambda x: x.shape)
shuffled_df.head()

X_train, X_test, y_train, y_test = train_test_split(shuffled_df['time_series'], shuffled_df['label'], train_size=.75, random_state=seed)

In [32]:
S, v_t_list_train = utils.compute_S_matrix(X_train)

In [33]:
_, v_t_list_test = utils.compute_S_matrix(X_test)

In [34]:
K_eros_train = utils.compute_kernel_matrix(len(X_train), weight_vector=w, v_t_list=v_t_list_train)
V = utils.perform_PCA(len(X_train), weight_vector=w, v_t_list=v_t_list_train)

K_eros_test = utils.compute_test_kernel_matrix(len(X_train), len(X_test), w, v_t_list_train, v_t_list_test)

In [35]:
N_test = len(X_test)
N_train = len(X_train)
O_test = np.ones(shape=(N_test, N_train))*(1/N_train)
O_train = np.ones(shape=(N_train, N_train))*(1/N_train)

K_eros_test_mc = K_eros_test - O_test@K_eros_train - K_eros_test@O_train + O_test@K_eros_train@O_train

Y = K_eros_test_mc @ V

the $i$-th MTS item in the test set is represented as features (length = num examples in the training set) in the $i$-th row of $\mathbf{Y}$

In [36]:
Y.shape

(75, 223)

In [43]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler

SC = StandardScaler()
SC.fit(V)
matrix_X_train_mean_fv_scaled = SC.transform(V)
matrix_X_test_mean_fv_scaled = SC.transform(Y)

prova_train = y_train.to_numpy().astype('int')
prova_test = y_test.to_numpy().astype('int')

svc = SVC()
svc.fit(matrix_X_train_mean_fv_scaled[:,:7], prova_train)

predictions = svc.predict(matrix_X_test_mean_fv_scaled[:,:7])

print(accuracy_score(prova_test, predictions))

0.14666666666666667
